In [ ]:
import pandas as pd
import dask.dataframe as dd
import csv
import statistics

In [ ]:
# Specify the columns to import
columns_to_import = ['CountyName', 'DwellingTypeDescr', 'Year_of_Construction', 'TypeofRating',
                     'EnergyRating', 'BerRating', 'GroundFloorArea(sq m)', 'FloorArea', 'NoStoreys',
                     'MainSpaceHeatingFuel', 'MainWaterHeatingFuel', 'HSMainSystemEfficiency',
                     'HSEffAdjFactor', 'HSSupplHeatFraction', 'HSSupplSystemEff', 'WHMainSystemEff',
                     'WHEffAdjFactor', 'DeclaredLossFactor', 'ThermalBridgingFactor', 'LivingAreaPercent',
                     'HESSchemeUpgrade', 'SA_Code','CO2Rating']

# Specify the data types for the columns
dtypes = {'DeclaredLossFactor': 'float64', 'SA_Code': 'object'}

In [ ]:
# Read the file and import specified columns, handling bad lines using on_bad_lines
data = dd.read_csv('BERPublicsearch.txt', sep='\t', encoding='latin1', usecols=columns_to_import, on_bad_lines='skip', quoting=csv.QUOTE_NONE, dtype=dtypes)

In [ ]:
# Compute the result
result = data.compute()

# Display the first few rows
print(result.head())

In [ ]:
# Shape of the file
result.shape

In [ ]:
# Create flags - Source https://www.sciencedirect.com/science/article/pii/S2352340920301414 (Appendix A. Supplementary data > Multimedia component 2.)
data['Flag_MainFloorArea_0'] = (data['GroundFloorArea(sq m)'] == 0).astype(int)
data['Flag_TotalFloorArea_less_equal_30'] = (data['GroundFloorArea(sq m)'] <= 30).astype(int)
data['Flag_ProvisionalRemoved'] = (data['TypeofRating'] == 'Provisional').astype(int)
data['Flag_TotalFloorArea_greater_than_1000'] = (data['GroundFloorArea(sq m)'] > 1000).astype(int)
data['Flag_Terraced_Homes_Apartments_MainFloorArea_greater_than_500'] = ((data['DwellingTypeDescr'].isin(['End of terrace house','Mid-terrace house','Top-floor apartment','Mid-floor apartment','Basement Dwelling','Apartment','Semi-detached house','Maisonette','Ground-floor apartment'])) & (data['GroundFloorArea(sq m)'] > 500)).astype(int)
data['Flag_HSMainSystemEfficiency_less_than_19'] = (data['HSMainSystemEfficiency'] < 19).astype(int)
data['Flag_HSEffAdjFactor_less_than_0.7'] = (data['HSEffAdjFactor'] < 0.7).astype(int)
data['Flag_WHMainSystemEfficiency_less_than_19_or_greater_than_450'] = ((data['WHMainSystemEff'] < 19) | (data['WHMainSystemEff'] > 450)).astype(int)
data['Flag_WHEffAdjFactor_less_than_0.7'] = (data['WHEffAdjFactor'] < 0.7).astype(int)
data['Flag_HSSupplSystemEfficiency_between_0_and_19'] = ((data['HSSupplSystemEff'] > 0) & (data['HSSupplSystemEff'] < 19)).astype(int)
data['Flag_LivingAreaPercent_outside_range'] = ((data['LivingAreaPercent'] > 90) | (data['LivingAreaPercent'] < 5)).astype(int)
data['Flag_HSSupplHeatFraction_not_in_range'] = (~data['HSSupplHeatFraction'].isin([0,0.1,0.15,0.2])).astype(int)
data['Flag_DeclaredLossFactor_greater_than_20'] = (data['DeclaredLossFactor'] > 20).astype(int)
data['Flag_ThermalBridgingFactor_outside_range'] = ((data['ThermalBridgingFactor'] < 0) | (data['ThermalBridgingFactor'] > 0.15)).astype(int)

In [ ]:
# Compute the result
result = data.compute()

# Display the first few rows
print(result.head())

In [ ]:
# Shape of the file
result.shape

In [ ]:
# Group by SA_Code and calculate count, sum, and average
summary_table = result.groupby('SA_Code').agg({'SA_Code': 'count', 'HESSchemeUpgrade': 'sum', 'BerRating': 'mean'})

In [ ]:
# Rename the columns for clarity
summary_table.rename(columns={'SA_Code': 'Count', 'HESSchemeUpgrade': 'Sum of HESSchemeUpgrade', 'BerRating': 'Average BerRating'}, inplace=True)

In [ ]:
# Display the summary table
print(summary_table)

In [ ]:
# Define the flags
flags = ['Flag_MainFloorArea_0', 'Flag_TotalFloorArea_less_equal_30', 'Flag_ProvisionalRemoved', 
         'Flag_TotalFloorArea_greater_than_1000', 'Flag_Terraced_Homes_Apartments_MainFloorArea_greater_than_500', 
         'Flag_HSMainSystemEfficiency_less_than_19', 'Flag_HSEffAdjFactor_less_than_0.7', 
         'Flag_WHMainSystemEfficiency_less_than_19_or_greater_than_450', 'Flag_WHEffAdjFactor_less_than_0.7', 
         'Flag_HSSupplSystemEfficiency_between_0_and_19', 'Flag_LivingAreaPercent_outside_range', 
         'Flag_HSSupplHeatFraction_not_in_range', 'Flag_DeclaredLossFactor_greater_than_20', 
         'Flag_ThermalBridgingFactor_outside_range']

In [ ]:
# Group by SA_Code and calculate count and sum for each flag
flag_summary = result.groupby('SA_Code')[flags].sum()

In [ ]:
# Merge with the original summary table
summary_table_with_flags = summary_table.merge(flag_summary, on='SA_Code')

In [ ]:
# Display the summary table with flags
print(summary_table_with_flags)

In [ ]:
# Export data to Excel
summary_table_with_flags.to_excel('.../summary_table_with_flags.xlsx', index=True)

In [ ]:
#INVESTIGATE DIFFERENCES IN ORIGINAL DATA VS TOTAL ENTRIES IN OUTPUT

In [ ]:
# Step 1: Check for missing values in the SA_CODE column of the original result DataFrame
missing_values = result['SA_Code'].isnull().sum()
print("Number of missing values in SA_Code column:", missing_values)

In [ ]:
# Step 2: Check for duplicates in the SA_CODE column of the original result DataFrame
duplicates = result['SA_Code'].duplicated().sum()
print("Number of duplicate values in SA_Code column:", duplicates)

In [ ]:
# Step 3: Compare the unique values in the SA_CODE column between the original result DataFrame and the grouped summary_table
unique_sa_codes_result = result['SA_Code'].unique()
unique_sa_codes_summary = summary_table.index.values
print("Number of unique SA_Code values in result DataFrame:", len(unique_sa_codes_result))
print("Number of unique SA_Code values in summary table:", len(unique_sa_codes_summary))

In [ ]:
result['TypeofRating'].value_counts()

In [ ]:
#IMPORT CSO DATA

In [ ]:
# Load the Excel file - connected to small area code
file_path = ".../CSO_DATA_2022.xlsx"
sheet_name = "Unpivoted"
data = pd.read_excel(file_path, sheet_name=sheet_name)

In [ ]:
# Filter data based on criteria
CSO_data = data[(data['Statistic Label'] == 'Private households') & (data['Type of Accommodation'] == 'Total') & (data['CSO Small Areas 2022'] != 'Ireland')]

In [ ]:
# Calculate the sum of the VALUE column
sum_of_value = CSO_data['VALUE'].sum()

In [ ]:
# Now CSO_data contains the desired data, and sum_of_value contains the sum of the VALUE column
print("Sum of VALUE:", sum_of_value)

In [ ]:
# Group CSO_data by 'CSO Small Areas 2022' (assuming this is the column containing SA_Code) and calculate the sum of 'VALUE'
sum_of_value_per_SA_Code = CSO_data.groupby('CSO Small Areas 2022')['VALUE'].sum().reset_index()

In [ ]:
# Reset the index of summary_table_with_flags
summary_table_with_flags.reset_index(inplace=True)

In [ ]:
# Set the index of summary_table_with_flags to 'SA_Code'
summary_table_with_flags.set_index('SA_Code', inplace=True)

In [ ]:
# Reset the index of sum_of_value_per_SA_Code
sum_of_value_per_SA_Code.reset_index(inplace=True)

In [ ]:
# Set the index of sum_of_value_per_SA_Code to 'CSO Small Areas 2022'
sum_of_value_per_SA_Code.set_index('CSO Small Areas 2022', inplace=True)

In [ ]:
# Merge the two dataframes on their indexes
merged_table = summary_table_with_flags.merge(sum_of_value_per_SA_Code, left_index=True, right_index=True, how='left')

In [ ]:
# Export data to Excel
merged_table.to_excel('.../summary_table_with_value.xlsx', index=True)

In [ ]:
print(merged_table.head())

In [ ]:
print(summary_table_with_flags.columns)

In [ ]:
print(summary_table_with_flags.index)

In [ ]:
# Define a function to map mean BerRating to mean EnergyRating
def map_to_ber_rating(mean_ber_rating):
    if pd.isnull(mean_ber_rating):
        return 'N/A'
    elif mean_ber_rating <= 25:
        return 'A1'
    elif mean_ber_rating <= 50:
        return 'A2'
    elif mean_ber_rating <= 75:
        return 'A3'
    elif mean_ber_rating <= 100:
        return 'B1'
    elif mean_ber_rating <= 125:
        return 'B2'
    elif mean_ber_rating <= 150:
        return 'B3'
    elif mean_ber_rating <= 175:
        return 'C1'
    elif mean_ber_rating <= 200:
        return 'C2'
    elif mean_ber_rating <= 225:
        return 'C3'
    elif mean_ber_rating <= 260:
        return 'D1'
    elif mean_ber_rating <= 300:
        return 'D2'
    elif mean_ber_rating <= 340:
        return 'E1'
    elif mean_ber_rating <= 360:
        return 'E2'
    elif mean_ber_rating <= 450:
        return 'F'
    else:
        return 'G'

In [ ]:
# Apply the function to create a new column 'BER_PCKG_AVE_ACTUAL_BER_RATING_CD'
summary_table['BER_PCKG_AVE_ACTUAL_BER_RATING_CD'] = summary_table['Average BerRating'].apply(map_to_ber_rating)

In [ ]:
# Merge summary_table with summary_table_with_flags based on index (SA_Code)
summary_table_with_flags = summary_table_with_flags.merge(summary_table[['BER_PCKG_AVE_ACTUAL_BER_RATING_CD']], left_index=True, right_index=True, how='left')

In [ ]:
# Filter result DataFrame where SA_Code is equal to '048059003'
filtered_result = result[result['SA_Code'] == '048059003']

# Display the first few rows of the filtered DataFrame
print(filtered_result.head())

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Step 1: Read Shapefile Data
shapefile_path = ".../CSO_Small_Areas_-_National_Statistical_Boundaries_-_2022_-_Ungeneralised/SMALL_AREA_2022.shp"
gdf = gpd.read_file(shapefile_path)

# Step 2: Read Excel Data from the SEAI_ALL_DATA tab
excel_file = "summary_table_with_value_20240229_v1.xlsx"
summary_df = pd.read_excel(excel_file, sheet_name="SEAI_ALL_DATA")

# Step 3: Merge Data
merged_df = gdf.merge(summary_df, left_on='SA_PUB2022', right_on='SA_Code', how='left')

# Step 4: Plot Choropleth Map

# Define custom colormap from red to green
colors = [(1, 0, 0), (1, 1, 0), (0, 1, 0)]  # Red to Yellow to Green
cmap = LinearSegmentedColormap.from_list("CustomRedGreen", colors, N=100)

fig, ax = plt.subplots(figsize=(10, 6))

# Plot the choropleth map with adjusted parameters
merged_df.plot(column='Percent_Availed_of_Scheme', cmap=cmap, vmin=0, vmax=0.15, linewidth=0.1, ax=ax, edgecolor='grey', legend=True)

# Set title and remove axis
ax.set_title('Percent Availed of Scheme')
ax.axis('off')

# Save the map as an image file
plt.savefig(".../SEAI_CSO_map.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
#DUBLIN ONLY

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Step 1: Read Shapefile Data
shapefile_path = ".../CSO_Small_Areas_-_National_Statistical_Boundaries_-_2022_-_Ungeneralised/SMALL_AREA_2022.shp"
gdf = gpd.read_file(shapefile_path)

# Filter data for County Dublin only
county_dublin_gdf = gdf[gdf['COUNTY_ENG'] == 'DUBLIN CITY']

# Step 2: Read Excel Data from the SEAI_ALL_DATA tab
excel_file = "summary_table_with_value_20240229_v1.xlsx"
summary_df = pd.read_excel(excel_file, sheet_name="SEAI_ALL_DATA")

# Step 3: Merge Data
merged_df = county_dublin_gdf.merge(summary_df, left_on='SA_PUB2022', right_on='SA_Code', how='left')

# Step 4: Plot Choropleth Map

# Define custom colormap from red to green
colors = [(1, 0, 0), (1, 1, 0), (0, 1, 0)]  # Red to Yellow to Green
cmap = LinearSegmentedColormap.from_list("CustomRedGreen", colors, N=100)

fig, ax = plt.subplots(figsize=(10, 6))

# Plot the choropleth map with adjusted parameters
merged_df.plot(column='Percent_Availed_of_Scheme', cmap=cmap, vmin=0, vmax=0.15, linewidth=0.1, ax=ax, edgecolor='grey', legend=True)

# Set title and remove axis
ax.set_title('Percent Availed of Scheme in County Dublin')
ax.axis('off')

# Save the map as an image file
plt.savefig(".../County_Dublin_Map.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# PPR GEOCODING #
import pandas as pd
import requests

def geocode_address(address):
    url = f'https://nominatim.openstreetmap.org/search?q={address}&format=json'
    response = requests.get(url, verify=False)  # Disabling SSL certificate verification
    print("Response content:", response.content)  # Print the response content
    data = response.json()

    if data:
        latitude = float(data[0]['lat'])
        longitude = float(data[0]['lon'])
        return latitude, longitude
    else:
        return None

# Load the CSV file into a pandas DataFrame
csv_file = r'...\PPR_ALL_March2024.csv'
data = pd.read_csv(csv_file, encoding='latin1')

# Rename the 'Date of Sale (dd/mm/yyyy)' column to 'Date of Sale'
data.rename(columns={'Date of Sale (dd/mm/yyyy)': 'Date of Sale'}, inplace=True)

# Sort the DataFrame based on 'Date of Sale' in descending order to get the most recent entries
data['Date of Sale'] = pd.to_datetime(data['Date of Sale'], format='%d/%m/%Y')
data.sort_values(by='Date of Sale', ascending=False, inplace=True)

# Limit the DataFrame to the most recent 2500 entries
data = data.head(2500)

# Iterate over each row in the DataFrame and geocode the address
for index, row in data.iterrows():
    address = row['Address']
    result = geocode_address(address)
    if result is not None:
        latitude, longitude = result
        data.at[index, 'Latitude'] = latitude
        data.at[index, 'Longitude'] = longitude
    else:
        data.at[index, 'Latitude'] = None
        data.at[index, 'Longitude'] = None

# Save the updated DataFrame with latitude and longitude to a new CSV file
output_file = r'...\PPR_ALL_March2024_geocoded.csv'
data.to_csv(output_file, index=False)

print("Geocoding completed. Results saved to:", output_file)

In [ ]:
#CHECK MATCHES BETWEEN SEAI AND BER CERT DATA#

import pandas as pd

# Read data from Excel file
ber_certs_data = pd.read_excel('.../BER_CERTS_March2024_V2.xlsx')

# Select required columns
ber_certs_subset = ber_certs_data[['BERNumber', 'EnergyRating', 'FloorArea', 'YearOfConstruction','ResearchMatchKey']]


# Concatenate values from existing columns to create ResearchMatchKey
result['ResearchMatchKey'] = (result['DwellingTypeDescr'] + '|' +
                              result['Year_of_Construction'].astype(str) + '|' +
                              result['GroundFloorArea(sq m)'].apply(lambda x: '{:.3f}'.format(x)) + '|' +
                              result['EnergyRating'].astype(str) + '|' +
                              result['BerRating'].apply(lambda x: '{:.3f}'.format(x)) + '|' +
                              result['CO2Rating'].apply(lambda x: '{:.2f}'.format(x)))

# Merge data with result DataFrame on matching values
merged_data = pd.merge(result, ber_certs_subset, how='inner',
                       left_on=['ResearchMatchKey'],
                       right_on=['ResearchMatchKey'])


# Now merged_data contains only the rows where EnergyRating, FloorArea, and YearOfConstruction match between the two datasets

# Create flags in merged_data
merged_data['Flag_MainFloorArea_0'] = (merged_data['GroundFloorArea(sq m)'] == 0).astype(int)
merged_data['Flag_TotalFloorArea_less_equal_30'] = (merged_data['GroundFloorArea(sq m)'] <= 30).astype(int)
merged_data['Flag_ProvisionalRemoved'] = (merged_data['TypeofRating'] == 'Provisional').astype(int)
merged_data['Flag_TotalFloorArea_greater_than_1000'] = (merged_data['GroundFloorArea(sq m)'] > 1000).astype(int)
merged_data['Flag_Terraced_Homes_Apartments_MainFloorArea_greater_than_500'] = (
(merged_data['DwellingTypeDescr'].isin(['End of terrace house', 'Mid-terrace house', 'Top-floor apartment', 'Mid-floor apartment', 'Basement Dwelling', 'Apartment', 'Semi-detached house', 'Maisonette', 'Ground-floor apartment'])) &
(merged_data['GroundFloorArea(sq m)'] > 500)
).astype(int)
merged_data['Flag_HSMainSystemEfficiency_less_than_19'] = (merged_data['HSMainSystemEfficiency'] < 19).astype(int)
merged_data['Flag_HSEffAdjFactor_less_than_0.7'] = (merged_data['HSEffAdjFactor'] < 0.7).astype(int)
merged_data['Flag_WHMainSystemEfficiency_less_than_19_or_greater_than_450'] = (
(merged_data['WHMainSystemEff'] < 19) | (merged_data['WHMainSystemEff'] > 450)
).astype(int)
merged_data['Flag_WHEffAdjFactor_less_than_0.7'] = (merged_data['WHEffAdjFactor'] < 0.7).astype(int)
merged_data['Flag_HSSupplSystemEfficiency_between_0_and_19'] = (
(merged_data['HSSupplSystemEff'] > 0) & (merged_data['HSSupplSystemEff'] < 19)
).astype(int)
merged_data['Flag_LivingAreaPercent_outside_range'] = (
(merged_data['LivingAreaPercent'] > 90) | (merged_data['LivingAreaPercent'] < 5)
).astype(int)
merged_data['Flag_HSSupplHeatFraction_not_in_range'] = (
~merged_data['HSSupplHeatFraction'].isin([0, 0.1, 0.15, 0.2])
).astype(int)
merged_data['Flag_DeclaredLossFactor_greater_than_20'] = (merged_data['DeclaredLossFactor'] > 20).astype(int)
merged_data['Flag_ThermalBridgingFactor_outside_range'] = (
(merged_data['ThermalBridgingFactor'] < 0) | (merged_data['ThermalBridgingFactor'] > 0.15)
).astype(int)

In [ ]:
# Specify the path where you want to save the merged data Excel file
output_file_path = '.../merged_data.xlsx'

# Save the merged data to an Excel file
merged_data.to_excel(output_file_path, index=False)

print("Merged data has been saved to:", output_file_path)